In [1]:
from tesimodules import Classificator as cs, Splitter
import pandas as pd
import numpy as np
import importlib
from sklearn.preprocessing import StandardScaler

source = 'https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data'

iris_df = pd.read_csv(
    filepath_or_buffer=source,
    header=None,
    sep=',')

iris_df.columns=['sepal_len', 'sepal_wid', 'petal_len', 'petal_wid', 'class']
iris_df.dropna(how="all", inplace=True) # drops the empty line at file-end

iris_values = iris_df.iloc[:,0:4].values
iris_labels = iris_df.iloc[:,4].values

iris_values_std = StandardScaler().fit_transform(iris_values)

In [7]:
model = cs.Model_PL(c=1, sigma=5, path="./model", model_name="iris-model#7")

train_values, train_labels, test_values, test_labels, train_indices, test_indices = Splitter.split_train_test_dataset(iris_values_std, iris_labels)

[ 58  99   0 104 133  52  26  59 129  64 123  74  83  44 105   8 125  89
 126 117  68 127  46 147  61  79   5  75   7  80  33  21 115 135  78  19
   6  93  30 149  42   3 138  57 137   1  72  48  27 132  20  92 128  37
 136  17  51  86 122 144 109 108  22 148  84   9  56  88  97 146  66   4
  45  77 120 101 141 116 114  38  23  70 121 111  25  41  81  67 106  71]
[130  32  55 119  29 145  65 102 110 107  13  40 112  10  35  96 140  62
 113  50  53  18  76 143  82  85  47 100  90  31  11  95  36 131 134  12
  54  98  91  16  49  28  69  34   2  73  87 103  60  63 139 124  15  43
  24  14  39 118 142  94]


In [8]:
model.fit(train_values, train_labels=list(zip(train_indices, train_labels)), mu=cs.Get_mu(iris_labels))

FUNCTION Iris-setosa:
 - MEMBERSHIP FUNCTION: <function LinearFuzzifier.get_fuzzified_membership.<locals>.estimated_membership at 0x114477a70> 
 - SECONDS: 5.79
 - MINUTES: 0.10
FUNCTION Iris-versicolor:
 - MEMBERSHIP FUNCTION: <function LinearFuzzifier.get_fuzzified_membership.<locals>.estimated_membership at 0x1a19578170> 
 - SECONDS: 5.48
 - MINUTES: 0.09
FUNCTION Iris-virginica:
 - MEMBERSHIP FUNCTION: <function LinearFuzzifier.get_fuzzified_membership.<locals>.estimated_membership at 0x1a19578c20> 
 - SECONDS: 5.73
 - MINUTES: 0.10


Model_PL(c=1, model_name='iris-model#7', path='./model', sigma=5)

In [9]:
model.score(test_values, test_labels, labels_indexed=False)

0.9666666666666667

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

kf = KFold(n_splits=3, shuffle=True)

clf = GridSearchCV(model, {'c':[1, 10000], 'sigma': [.1, .25, .225, .5,]}, cv=kf, n_jobs=-1, verbose=10)
clf.fit(train_values, list(zip(train_indices, train_labels)), mu=cs.Get_mu(iris_labels))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed:   38.8s
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:   56.7s
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:  1.6min
[Parallel(n_jobs=-1)]: Done  20 out of  24 | elapsed:  1.6min remaining:   19.0s
[Parallel(n_jobs=-1)]: Done  24 out of  24 | elapsed:  1.9min finished
FUNCTION Iris-setosa:
 - MEMBERSHIP FUNCTION: <function LinearFuzzifier.get_fuzzified_membership.<locals>.estimated_membership at 0x1a195789e0> 
 - SECONDS: 5.17
 - MINUTES: 0.09
FUNCTION Iris-versicolor:
 - MEMBERSHIP FUNCTION: <function LinearFuzzifier.get_fuzzified_membership.<locals>.estimated_membership at 0x1a19554290> 
 - SECONDS: 4.53
 - MINUTES: 0.08
FUNCTION Iris-virginica:
 - MEMBERSHIP FUNCTION: <function LinearFuzzifier.get_fuzzified_membership.<locals>.estimated_membership at 0x1a19554320> 
 - SECONDS: 4.83

GridSearchCV(cv=KFold(n_splits=3, random_state=None, shuffle=True),
             error_score='raise-deprecating',
             estimator=Model_PL(c=1, model_name='iris-model#7', path='./model',
                                sigma=5),
             iid='warn', n_jobs=-1,
             param_grid={'c': [1, 10000], 'sigma': [0.1, 0.25, 0.225, 0.5]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=10)

In [16]:
clf.cv_results_

{'mean_fit_time': array([18.00530505, 16.82060512, 16.91378808, 17.50713062, 19.64899651,
        18.02194802, 17.92449665, 18.35595655]),
 'std_fit_time': array([0.0803537 , 0.68036478, 0.52449282, 0.02632895, 0.08297109,
        1.13382451, 0.44710204, 0.08206828]),
 'mean_score_time': array([0.27825761, 0.28243534, 0.32209293, 0.31688436, 0.34524401,
        0.29349136, 0.25936937, 0.22443382]),
 'std_score_time': array([0.0094624 , 0.00850469, 0.03478446, 0.03966733, 0.05221296,
        0.02619739, 0.00656227, 0.04316553]),
 'param_c': masked_array(data=[1, 1, 1, 1, 10000, 10000, 10000, 10000],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'param_sigma': masked_array(data=[0.1, 0.25, 0.225, 0.5, 0.1, 0.25, 0.225, 0.5],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'c': 1, 'sigma': 0.1},
  {'c': 1, 'sigma'

In [17]:
clf.best_index_

1

In [2]:
from tesimodules.ModelSelection import GridSearchCV as gs

clf = gs(model, )